# 1 - Voxelise geological model

Import all CSV files and interpolate each surface to a predefined grid over which to solve Darcy flow and heat flow.

In [ ]:
import numpy as np
import stripy
import matplotlib.pyplot as plt
from scipy import interpolate
from scipy.spatial import cKDTree
import cartopy.crs as ccrs
import os
import conduction

%matplotlib inline

In [ ]:
data_dir = "../Data/"

xmin, xmax = 1e99, 1e-99
ymin, ymax = 1e99, 1e-99
zmin, zmax = 1e99, 1e-99

csvfiles = []
for i, f in enumerate(sorted(os.listdir(data_dir))):
    if f.endswith('.csv') and f[0:2].isdigit():
        csvfiles.append(f)
        
        xyz = np.loadtxt(data_dir+f, delimiter=',', skiprows=1)
        
        xmin, ymin, zmin = np.minimum([xmin, ymin, zmin], xyz.min(axis=0))
        xmax, ymax, zmax = np.maximum([xmax, ymax, zmax], xyz.max(axis=0))
        
        print("{:35} : av depth {:10.3f} +- {:9.3f} metres".format(f, xyz[:,-1].mean(), np.std(xyz[:,-1])))

csvfiles = list(sorted(csvfiles))

print(" ")
print("x {:8.3f} -> {:8.3f}".format(xmin/1e3,xmax/1e3))
print("y {:8.3f} -> {:8.3f}".format(ymin/1e3,ymax/1e3))
print("z {:8.3f} -> {:8.3f}".format(zmin/1e3,zmax/1e3))


## Export high resolution grids

Save to a numpy zip archive

In [ ]:
Nx, Ny = 791, 1906

Xcoords = np.linspace(xmin, xmax, Nx)
Ycoords = np.linspace(ymin, ymax, Ny)

xq, yq = np.meshgrid(Xcoords, Ycoords)
xq_ = xq.ravel()
yq_ = yq.ravel()

In [ ]:
z_grid_prev = np.full((Ny,Nx), zmax)

grid_list = []

for lith, f in enumerate(csvfiles):

    # load the surface and remove duplicates
    x,y,z = np.loadtxt(data_dir+f, delimiter=',', skiprows=1, unpack=True)
    unique_xy, unique_index = np.unique(np.c_[x,y], axis=0, return_index=True)
    x, y = list(unique_xy.T)
    z = z[unique_index]

    # interpolate to grid
    mesh = stripy.Triangulation(x, y, tree=True, permute=True)
    mesh.update_tension_factors(z)
    z_grid = mesh.interpolate_to_grid(Xcoords, Ycoords, z)
    z_near, zierr = mesh.interpolate_nearest(xq_, yq_, z)
    z_near = z_near.reshape(z_grid.shape)

    # set entries outside tolerance to nearest neighbour interpolant
    ztol = 0.1*(np.percentile(z,99) - np.percentile(z,1))
    z_mask = np.logical_or(z_grid > z_near + ztol, z_grid < z_near - ztol)
    z_grid[z_mask] = z_near[z_mask]

    # avoid the gap - make sure we interpolate where we have data
    d, idx = mesh.nearest_vertices(xq_, yq_)
    dtol = np.sqrt(mesh.areas()).mean()
    z_inv_mask = np.logical_or(zierr==1, d > dtol)
    z_grid.flat[z_inv_mask] = z_grid_prev.flat[z_inv_mask]

    # make sure no layer has negative thickness
    z_grid = np.minimum(z_grid, z_grid_prev)

    # store for next surface
    z_grid_prev = z_grid.copy()
    
    grid_list.append(z_grid.copy())
    
    print("finished processing {}".format(f))

grid_list = np.array(grid_list)

In [ ]:
## Save to npz archive

np.savez_compressed(data_dir+"sydney_basin_surfaces.npz", layers=np.array(grid_list),
                                                          Xcoords=Xcoords,
                                                          Ycoords=Ycoords)

## Create surface maps

Plot the undulation of each geological surface

In [ ]:
extent_local = [xmin,xmax,ymin,ymax]


fig, axes = plt.subplots(5,4, sharex=True, sharey=True, figsize=(14,35))

i = 0
for rax in axes:
    for ax in rax:
        x,y,z = np.loadtxt(data_dir+csvfiles[i], delimiter=',', skiprows=1, unpack=True)
        
        im = ax.scatter(x,y,c=z, cmap='terrain', vmin=-5e3, vmax=1e3)
        fig.colorbar(im, ax=ax, shrink=0.4)
        ax.set_title(csvfiles[i])
        
        i += 1
        
        if i == len(grid_list):
            break

In [ ]:
extent_local = [xmin,xmax,ymin,ymax]


fig, axes = plt.subplots(5,4, sharex=True, sharey=True, figsize=(14,35))

i = 0
for rax in axes:
    for ax in rax:
        im = ax.imshow(grid_list[i], extent=extent_local, origin='lower', cmap='terrain', vmin=-5e3, vmax=1e3)
        fig.colorbar(im, ax=ax, shrink=0.4)
        ax.set_title(csvfiles[i])
        
        i += 1
        
        if i == len(grid_list):
            break

In [ ]:
# create thickness maps

# dzgrid_list = np.diff(np.array(grid_list), axis=0)
grid_prev = np.full_like(grid_list[0], zmax)

fig, axes = plt.subplots(5,4, sharex=True, sharey=True, figsize=(14,35))

i = 0
for rax in axes:
    for ax in rax:
        dzgrid = grid_list[i] - grid_prev
        
        im = ax.imshow(dzgrid, extent=extent_local, origin='lower', cmap='terrain')
        fig.colorbar(im, ax=ax, shrink=0.4)
        ax.set_title(csvfiles[i])
        
        grid_prev = grid_list[i]
        i += 1
        
        if i == len(grid_list):
            break

## Now load geological model

In a parallel-safe way!

In [ ]:
## setup model resolution

# global size
Nx, Ny, Nz = 101, 101, 201

# get local grid
solver = conduction.ConductionND((xmin,ymin,zmin), (xmax,ymax,zmax), (Nx,Ny,Nz))
Xcoords, Ycoords, Zcoords = solver.grid_coords
nx, ny, nz = Xcoords.size, Ycoords.size, Zcoords.size

tree = cKDTree(solver.coords)

xq, yq = np.meshgrid(Xcoords, Ycoords)

voxel_model = np.full((nz,ny,nx), -1, dtype=np.int)
layer_mask = np.zeros(nz*ny*nx, dtype=bool)

In [ ]:
with np.load(data_dir+"sydney_basin_surfaces.npz", "r") as npz:
    grid_list    = npz["layers"]
    grid_Xcoords = npz['Xcoords']
    grid_Ycoords = npz['Ycoords']


n_layers = grid_list.shape[0]

# set up interpolation object
interp = interpolate.RegularGridInterpolator((grid_Ycoords, grid_Xcoords), grid_list[0], method="linear")

for lith in range(n_layers):
    interp.values = grid_list[lith]
    z_grid = interp((yq,xq))
    
    # interpolate to voxel_model
    d, idx = tree.query(np.c_[xq.ravel(), yq.ravel(), z_grid.ravel()])
    layer_mask.fill(0)
    layer_mask[idx] = True
    i0, j0, k0 = np.where(layer_mask.reshape(nz,ny,nx))
    for i in range(i0.size):
        voxel_model[:i0[i], j0[i], k0[i]] = lith+1


__Save the geological model to HDF5__

In [ ]:
h5_output = "sydney_basin.h5"

solver.save_mesh_to_hdf5(h5_output)
solver.save_field_to_hdf5(h5_output, lithology=voxel_model.ravel())
conduction.tools.generateXdmf(h5_output)

__Testing__

In [ ]:
with np.load(data_dir+"sydney_basin_surfaces.npz", "r") as npz:
    grid_list    = npz["layers"]
    grid_Xcoords = npz['Xcoords']
    grid_Ycoords = npz['Ycoords']

# set up interpolation object
interp = interpolate.RegularGridInterpolator((grid_Ycoords, grid_Xcoords), grid_list[0], method="linear")

# update grid list for top and bottom of model
grid_list = list(grid_list)
grid_list.append(np.full_like(grid_list[0], zmin))
grid_list = np.array(grid_list)

n_layers = grid_list.shape[0]

# set up interpolation object
interp = interpolate.RegularGridInterpolator((grid_Ycoords, grid_Xcoords), grid_list[0], method="linear")


cell_centroid = (1092810.4463113246, 6022927.350231495, -5607.06948146214)
cx, cy, cz = cell_centroid

for lith in range(n_layers):
    interp.values = grid_list[lith]
    z_interp = interp((cy,cx))
    
    print("{:2} {:10.2f}, {:2f}".format(lith, z_interp, cz))

    # starting from surface and going deeper with each layer
    if cz > z_interp:
        print("stop at {}".format(lith))
#         break



In [ ]:
d, idx = tree.query(cell_centroid)
print(voxel_model.flat[idx])

layer_mask.fill(0)
layer_mask[idx] = True
i0, j0, k0 = np.where(layer_mask.reshape(nz,ny,nx))
voxel_model[:,j0,k0]